# Load Packages and start clients

In [1]:
import os
import csv
import base64
from openai import OpenAI
import anthropic
import httpx
import json
import re
from config import ANTHROPIC_API_KEY, OPENAI_API_KEY

openai_client = OpenAI(api_key=OPENAI_API_KEY)

anthropic_client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

# Load files and create dictionaries

In [2]:
# Carpeta con las imágenes de las preguntas
image_folder = "tests/mat-m1/questions-images"

# Archivo CSV con las respuestas correctas
csv_file = "tests/mat-m1/answers.csv"

# Diccionarios para almacenar los resultados de cada modelo de IA
openai_answers = {}
openai_results = {}
anthropic_answers = {}

# Read the CSV file and store the correct answers in a dictionary. Skip first row (header)
correct_answers = {}
with open(csv_file, "r") as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        question_number, correct_answer = row
        correct_answers[question_number] = correct_answer

# Preparar el prompt

In [3]:
prompt = "Eres un experto en matemáticas. Mira detenidamente la imagen y dime cuál es la alternativa correcta. Asegúrate de no caer en alternativas distractoras (aquellas que se ven correctas pero no lo son). Resuelve esto paso a paso para asegurarte de que tienes la respuesta correcta. Cuando tengas la respuesta correcta, escríbela entre &&&. Por ejemplo, si la respuesta es la alternativa E, escribe 'La alternativa correcta es &&&E&&&'."

# Save JSON safe prompt
prompt = json.dumps(prompt)

# Give images to OpenAI

In [4]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Loop through each correct answer, only the first 5 items
for question_number, correct_answer in list(correct_answers.items()):
    # Get the image path
    image_path = os.path.join(image_folder, f"{question_number}.png")

    # Getting the base64 string
    base64_image = encode_image(image_path) 

    # Send the image to OpenAI Vision API
    openai_response = openai_client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
        "role": "user",
        "content": [
            {"type": "text", "text": f"{prompt}"},
            {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}",
            },
            },
        ],
        }
    ]
    )
    response = str(openai_response.choices[0].message.content)

    # Find first ocurrence of &&& in the response using find()
    start = response.find("&&&")

    # Get the next character after the first &&&
    openai_answer = response[start+3]
    
    # Compare the answers with the correct answer
    openai_answers[question_number] = openai_answer 

    openai_results[question_number] = openai_answer == correct_answer

# Save openai_results and openai_answers to a csv
with open("openai_results.csv", "w") as file:
    writer = csv.writer(file)
    writer.writerow(["Question", "Answer", "Correct"])
    for question, answer in openai_answers.items():
        writer.writerow([question, answer, openai_results[question]])

In [ ]:
# Loop through each correct answer
for question_number, correct_answer in correct_answers.items():
    # Get the image path
    image_path = os.path.join(image_folder, f"{question_number}.png")

    # Send the image to OpenAI Vision API
    response = openai_client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
        "role": "user",
        "content": [
            {"type": "text", "text": "What’s in this image?"},
            {
            "type": "image_url",
            "image_url": {
                "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
            },
            },
        ],
        }
    ],
    max_tokens=300,
    )
    openai_answer = response["output"]["label"]

    # Send the image to Claude API of Anthropic
    with open(image_path, "rb") as file:
        image_base64 = base64.b64encode(file.read()).decode("utf-8")
    response = anthropic_client.image.create(
        image=image_base64,
        model="claude-vision-model-id"
    )
    anthropic_answer = response["choices"][0]["text"].strip()

    # Compare the answers with the correct answer
    openai_results[question_number] = openai_answer == correct_answer
    anthropic_results[question_number] = anthropic_answer == correct_answer

In [ ]:
# Procesar cada imagen de pregunta
for image_file in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_file)
    question_number = os.path.splitext(image_file)[0]  # Obtener el número de pregunta del nombre del archivo

    # Enviar la imagen a OpenAI Vision API
    response = openai.Image.create(
        file=image_path,
        model="image-classifier-v3"
    )

    # Enviar la imagen a Claude API de Anthropic
    response = anthropic.Image.create(
        image=image_path,
        model="claude-vision-model-id"
    )
    anthropic_answer = response["choices"][0]["text"].strip()  # Obtener la respuesta de Claude

    # Comparar las respuestas con la respuesta correcta
    correct_answer = correct_answers[question_number]
    openai_results[question_number] = openai_answer == correct_answer
    anthropic_results[question_number] = anthropic_answer == correct_answer

# Calcular el rendimiento de cada modelo de IA
openai_correct = sum(openai_results.values())
openai_total = len(openai_results)
openai_accuracy = openai_correct / openai_total

anthropic_correct = sum(anthropic_results.values())
anthropic_total = len(anthropic_results)
anthropic_accuracy = anthropic_correct / anthropic_total

# Imprimir los resultados
print("OpenAI Vision API:")
print(f"Correctas: {openai_correct}/{openai_total}")
print(f"Precisión: {openai_accuracy:.2%}")

print("Anthropic Claude API:")
print(f"Correctas: {anthropic_correct}/{anthropic_total}")
print(f"Precisión: {anthropic_accuracy:.2%}")

In [ ]:
image1_url = "https://upload.wikimedia.org/wikipedia/commons/a/a7/Camponotus_flavomarginatus_ant.jpg"
image1_media_type = "image/jpeg"
image1_data = base64.b64encode(httpx.get(image1_url).content).decode("utf-8")

message = anthropic_client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": image1_media_type,
                        "data": image1_data,
                    },
                },
                {
                    "type": "text",
                    "text": "Describe this image."
                }
            ],
        }
    ],
)
print(message)

In [ ]:
response = openai_client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "What’s in this image?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])